# Parse LSDYNA file to extract particle coordinate, type, Von Mises Stree

In [3]:
import re
import numpy as np

LOADING_PARTICLES = []
SUPPORT_PARTICLES = []

def parse_simulation(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    # Find all "particle position" lines and "plastic strain" lines using key words
    pos_lines_start, pos_lines_end = [], []
    strain_lines_start, strain_lines_end = [], []
    for idx, line in enumerate(lines):
        if line.startswith("*NODE"):
            pos_lines_start.append(idx)
        elif line.startswith("$NODAL_RESULTS"):  # $NODAL_RESULTS,(1d) *INITIAL_VELOCITY_NODE(2d)
            pos_lines_end.append(idx)
        elif line.startswith("$RESULT OF Effective Stress (v-m)"):
            strain_lines_start.append(idx)
        elif line.startswith("*END"):  
            strain_lines_end.append(idx)
            
    # Extact particle positions 
    trajectory = []
    for line_start, line_end in zip(pos_lines_start, pos_lines_end):
        pos_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        timestep = []
        for line in pos_lines:
            num_str = re.findall(r'[-\d\.e+]+', line)  # Regular expression findign scitific numbers
            (x, y) = (float(num_str[1]), float(num_str[2]))
            timestep.append((x,y))
        trajectory.append(timestep) 
    
    # Extact particle types
    particle_types = []
    pos_lines = lines[pos_lines_start[0]+1:pos_lines_end[0]]
    for line in pos_lines:
        num_str = re.findall(r'[-\d\.e+]+', line)
        if int(num_str[0]) in LOADING_PARTICLES:
            particle_types.append(3)   # kinematic particles
        elif int(num_str[0]) in SUPPORT_PARTICLES:
            particle_types.append(2)   # boundary particles (rigid)
        else:
            particle_types.append(1)   # normal concrete particles
    
    # Extrac Von Mises Stress
    strains = []
    for line_start, line_end in zip(strain_lines_start, strain_lines_end):
        strain_lines = lines[line_start+1:line_end]   # lines that contains positions in one time step
        strains_one_step = []
        for line in strain_lines:
            num_str = re.findall(r'[-+\d\.Ee]+', line)  # the expression matches one or more repetitions of "-", "integer", ".", "E",
            num = float(num_str[1])
            strains_one_step.append(num)
        strains.append(strains_one_step)     
    

    return np.array(trajectory).astype(np.float), np.array(particle_types).astype(np.float), np.array(strains).astype(np.float)

# Read Text and write to individual npz

In [ ]:
import os
import glob
import json
import random
import numpy as np

in_dir = f'/home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/Concrete2D-T/LS-DYNA-Output/'
out_dir = f'/home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/Concrete2D-T/LS-DYNA-Output/npz'

# Create out_dir if it does not exist
os.makedirs(out_dir, exist_ok=True)

simulations = glob.glob(in_dir + '*')
random.shuffle(simulations)
simulations.sort()

for idx, simulation in enumerate(simulations):
    print(f"{idx} Reading {simulation}...")
    
    trajectory_name = simulation.split('/')[-1]
    
    # Parse simulation data (ensure this function is defined)
    positions, particle_types, strains = parse_simulation(simulation)

    # Define output file path
    npz_path = os.path.join(out_dir, f"{trajectory_name}.npz")

    # Save the NumPy arrays in an .npz file
    np.savez(npz_path, positions=positions, particle_types=particle_types, strains=strains)

    print(f"Saved {npz_path}")

# Pre-process and write to npz for GNN training and testing

In [6]:
import glob
import json
import random
import numpy as np
import os

# Varianbles to be specified
DATASET = 'Concrete2D-T-Step2'
STEP_SIZE = 2 # downsample the timestep from the numerical model
TOTAL_STEP = 100

val_set = ['60-120', '80-140', '100-180']
test_set = ['60-130', '80-150', '100-170']

# Input and Output directory
in_dir = f'/home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/Concrete2D-T/npz/'
out_dir = f'/home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/{DATASET}/'

# Create out_dir if it does not exist
os.makedirs(out_dir, exist_ok=True)

## Normalisation parameters
pos_max, pos_min = np.array([100, 50]), np.array([-2.5, -50])
#strain_mean, strain_std = 143.09920564186177, 86.05175002337249  # Step 1, vms stats pre-computed from data
strain_mean, strain_std = 150.25897834554772, 83.50737010164767  # Step 2

simulations = glob.glob(in_dir + '*')
random.shuffle(simulations)
simulations.sort()
ds_train, ds_valid, ds_test = {}, {}, {}
vels = np.array([]).reshape(0, 2)
accs = np.array([]).reshape(0, 2)
strain_stats = np.array([])
train_info, valid_info, test_info = [], [], []

for idx, simulation in enumerate(simulations):
    print(f"{idx} Reading {simulation}...")
    
    trajectory_name = simulation.split('/')[-1]
    
    # Load data from npz
    with np.load(simulation) as data:
        positions = data['positions']
        particle_types = data['particle_types']
        strains = data['strains']
        print(positions.shape, particle_types.shape, strains.shape)
       
    # Identify the time instance when structure response initialise
    mean_strain = strains.mean(axis=1)
    first_nonzero_step_idx = next((i for i, x in enumerate(mean_strain) if x), None) # x!= 0 for strict match
    
    # Preprocessing positions
    positions = positions[first_nonzero_step_idx-1:first_nonzero_step_idx-1+TOTAL_STEP:STEP_SIZE,:-4:1,:] 
    positions = (positions - pos_min) / (pos_max - pos_min)  # Normalize based on overall min and max of all simulations
    # y_scalling_factor = (pos_max - pos_min)[0] / (pos_max - pos_min)[1]
    # positions[:,:,1] = positions[:,:,1] / y_scalling_factor   
    
    # Change the last 4 particle to boundary particle
    particle_types = particle_types[:-4]

    # Preprocessing strains
    strains = strains[first_nonzero_step_idx-1:first_nonzero_step_idx-1+TOTAL_STEP:STEP_SIZE, :-4:1]
    strains = (strains - strain_mean) / strain_std   ## standardize based on overall mean and std
    # strain_stats = np.concatenate((strain_stats, strains.flatten()), axis=0) # debug stats
    
    print(f"Position min:{positions.min(axis=(0,1))}, max:{positions.max(axis=(0,1))}")
    print(f"Strain min:{strains.min(axis=(0,1))}, max:{strains.max(axis=(0,1))}")
    print(f"Position shape:{positions.shape}, type shape:{particle_types.shape}, strain shape:{strains.shape}")
    print(f"Unique particle types: {np.unique(particle_types)}")
    
    # Data splits: train, valid, test
    key = trajectory_name
    trajectory_data = (positions, particle_types, strains)

    if any(name in trajectory_name for name in val_set):
        ds_valid[key] = trajectory_data
        valid_info.append(trajectory_name)
    elif any(name in trajectory_name for name in test_set):
        ds_test[key] = trajectory_data
        test_info.append(trajectory_name)
    else:
        ds_train[key] = trajectory_data
        train_info.append(trajectory_name)
        
    # Extract Vel and Acc statistics
    # positions of shape [timestep, particles, dimensions]
    vel_trajectory = positions[1:,:,:] - positions[:-1,:,:]
    acc_trajectory = vel_trajectory[1:,:,:]- vel_trajectory[:-1,:,:]
    
    vels = np.concatenate((vels, vel_trajectory.reshape(-1, 2)), axis=0)
    accs = np.concatenate((accs, acc_trajectory.reshape(-1, 2)), axis=0)
    
#print('strain_stats:', strain_stats.mean(), strain_stats.std())

vel_mean = list(vels.mean(axis=0))
vel_std = list(vels.std(axis=0))
acc_mean = list(accs.mean(axis=0))
acc_std = list(accs.std(axis=0))

# Save the entire dictionary under one key:
train_filepath = os.path.join(out_dir, 'train.npz')
valid_filepath = os.path.join(out_dir, 'valid.npz')
test_filepath = os.path.join(out_dir, 'test.npz')

np.savez(train_filepath, trajectories=ds_train)
np.savez(valid_filepath, trajectories=ds_valid)
np.savez(test_filepath, trajectories=ds_test)

print(f"{len(ds_train)} trajectories parsed and saved to train.npz.")
print(f"{len(ds_valid)} trajectories parsed and saved to valid.npz.")
print(f"{len(ds_test)}  trajectories parsed and saved to test.npz.")


# Save meta data
out_file = out_dir + 'metadata.json'

meta_data = {}
meta_data['dim'] = positions.shape[-1]
meta_data['sequence_length'] = positions.shape[0]
meta_data['dt'] = 0.002 * STEP_SIZE
meta_data['bounds'] = [[0, 1], [0, 1]]
meta_data['default_connectivity_radius'] = 0.03  # will be reset in train.py
meta_data['vel_mean'] = vel_mean
meta_data['vel_std'] = vel_std
meta_data['acc_mean'] = acc_mean
meta_data['acc_std'] = acc_std
meta_data['file_train'] = train_info
meta_data['file_valid'] = valid_info
meta_data['file_test'] = test_info
print(meta_data)

with open(out_file, 'w') as f:
    json.dump(meta_data, f)

0 Reading /home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/Concrete2D-T/npz/T-20-100-100.npz...
(152, 8004, 2) (8004,) (152, 8004)
Position min:[0.00208806 0.27246548], max:[0.97615396 0.72739373]
Strain min:-1.7993499036390201, max:2.112844907457707
Position shape:(50, 8000, 2), type shape:(8000,), strain shape:(50, 8000)
Unique particle types: [1.]
1 Reading /home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/Concrete2D-T/npz/T-20-100-110.npz...
(152, 8004, 2) (8004,) (152, 8004)
Position min:[0.00202606 0.25081747], max:[0.97610082 0.74929665]
Strain min:-1.7993499036390201, max:2.1531946394142825
Position shape:(50, 8000, 2), type shape:(8000,), strain shape:(50, 8000)
Unique particle types: [1.]
2 Reading /home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/Concrete2D-T/npz/T-20-100-120.npz...
(152, 8004, 2) (8004,) (152, 8004)
Position min:[0.00200911 0.23018063], max:[0.97651833 0.76993527]
Strain min:-1.7993499036390201, max:2.3594650557743364
Position shape:(50, 8000, 2), type shape:(8000,), str

In [5]:
print(particle_types.shape)
print(positions.shape)
print(strains.shape)

(6400,)
(50, 6400, 2)
(50, 6400)


In [6]:
import numpy as np

# Load the npz file
data = np.load('/home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/test/train.npz', allow_pickle=True)

# Extract the stored dictionary of trajectories
ds_train = data['trajectories'].item()

# Convert the dictionary items to a list of (key, value) tuples
train_data_list = list(ds_train.items())

# Get the first trajectory (tuple: key, value)
first_key, first_trajectory = train_data_list[0]

# Extract positions, particle types, and strains
positions, particle_types, strains = first_trajectory  # Unpack the tuple

# Print details
print(f"First trajectory key: {first_key}")
print(f"Positions shape: {positions.shape}")
print(f"Particle types shape: {particle_types.shape}")
print(f"Strains shape: {strains.shape}")


First trajectory key: T-20-100-100.npz
Positions shape: (50, 8000, 2)
Particle types shape: (8000,)
Strains shape: (50, 8000)


In [20]:
path = '/home/jovyan/share/EPIMETHEUS-LOCAL/gns_data/test/train.npz'
data = [item for _, item in np.load(path, allow_pickle=True)['trajectories'].item().items()]

print(data[0][0].shape)

(50, 8000, 2)


In [22]:
data[0]

(array([[[0.00542225, 0.4025    ],
         [0.00542225, 0.4075    ],
         [0.00542225, 0.4125    ],
         ...,
         [0.97615396, 0.5875    ],
         [0.97615396, 0.5925    ],
         [0.97615396, 0.5975    ]],
 
        [[0.00486922, 0.39726066],
         [0.0048349 , 0.40308638],
         [0.00483923, 0.40879182],
         ...,
         [0.97219575, 0.5875    ],
         [0.97219575, 0.5925    ],
         [0.97219575, 0.5975    ]],
 
        [[0.00486135, 0.39063412],
         [0.0048212 , 0.39672863],
         [0.00483605, 0.40283452],
         ...,
         [0.96834282, 0.5875    ],
         [0.96834282, 0.5925    ],
         [0.96834282, 0.5975    ]],
 
        ...,
 
        [[0.0186732 , 0.27495067],
         [0.01693533, 0.2808313 ],
         [0.01537509, 0.2869507 ],
         ...,
         [0.84884107, 0.587498  ],
         [0.84884309, 0.59249799],
         [0.84884509, 0.597498  ]],
 
        [[0.01913446, 0.27367611],
         [0.01735625, 0.27954442],
       